<a href="https://colab.research.google.com/github/Jaimemorillo/ShouldIwatchThisMovie/blob/master/memoria_encoding_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')

### Bibliografy
- https://towardsdatascience.com/text-encoding-a-review-7c929514cccf

- https://realpython.com/python-keras-text-classification/

## Sentences

In [ ]:
corpus = [
    'Este es el primer documento.',
    'Este documento es el segundo documento.',
    'Este es el tercero.',
    '¿Es este el primer documento?'
    ]

## Bag of words (not ordered)
Sentence as vector


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [ ]:
vectorizer.get_feature_names()

['documento', 'el', 'es', 'este', 'primer', 'segundo', 'tercero']

In [ ]:
X.toarray()

array([[1, 1, 1, 1, 1, 0, 0],
       [2, 1, 1, 1, 0, 1, 0],
       [0, 1, 1, 1, 0, 0, 1],
       [1, 1, 1, 1, 1, 0, 0]])

In [ ]:
pd.DataFrame(columns=vectorizer.get_feature_names(), data=X.toarray(), index=corpus)

,documento,el,es,este,primer,segundo,tercero
Este es el primer documento.,1,1,1,1,1,0,0
Este documento es el segundo documento.,2,1,1,1,0,1,0
Este es el tercero.,0,1,1,1,0,0,1
¿Es este el primer documento?,1,1,1,1,1,0,0


## One-Hot Encoding (ordered)
Word as vector

In [ ]:
words = vectorizer.get_feature_names()
words

['documento', 'el', 'es', 'este', 'primer', 'segundo', 'tercero']

In [ ]:
word_to_vector = {
    'documento' : [1, 0, 0, 0, 0, 0, 0],
    'el' : [0, 1, 0, 0, 0, 0, 0],
    'es' : [0, 0, 1, 0, 0, 0, 0],
    'este' : [0, 0, 0, 1, 0, 0, 0],
    'primer' : [0, 0, 0, 0, 1, 0, 0],
    'segundo' : [0, 0, 0, 0, 0, 1, 0],
    'tercero' : [0, 0, 0, 0, 0, 0, 1],
}

In [ ]:
corpus_as_array = [x.replace(".", "").replace("¿","").replace("?","").lower().split(" ") for x in corpus]

In [ ]:
corpus_as_array

[['este', 'es', 'el', 'primer', 'documento'],
 ['este', 'documento', 'es', 'el', 'segundo', 'documento'],
 ['este', 'es', 'el', 'tercero'],
 ['es', 'este', 'el', 'primer', 'documento']]

In [ ]:

one_hot_corpus = np.array([np.array([word_to_vector[x] for x in corpus_as_array[i]]) for i in range(0, len(corpus_as_array))])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


A cada palabra se le asigna un vector, la manera más sencilla es asignarle un vector que contenga 0's en todas las posiciones menos un 1 en la posición que corresponde con su índice en el diccionario.

In [ ]:
#@title
df_word_to_vector = pd.DataFrame(pd.Series(vectorizer.vocabulary_).sort_values(), columns=['indice'])
df_word_to_vector['vector'] = pd.Series(word_to_vector)
df_word_to_vector

,indice,vector
documento,0,"[1, 0, 0, 0, 0, 0, 0]"
el,1,"[0, 1, 0, 0, 0, 0, 0]"
es,2,"[0, 0, 1, 0, 0, 0, 0]"
este,3,"[0, 0, 0, 1, 0, 0, 0]"
primer,4,"[0, 0, 0, 0, 1, 0, 0]"
segundo,5,"[0, 0, 0, 0, 0, 1, 0]"
tercero,6,"[0, 0, 0, 0, 0, 0, 1]"


Hay que aplicarle la codificación anterior a cada una de las oraciones. Primero necesitamos llevar la frase a formato array separando cada una de las palabras. Y ya podemos mapear cada una de las palabras a su vector correspondiente.

Este es el resultado:

In [ ]:
#@title
data = {
    'sentence_as_array': corpus_as_array,
    'one_hot_sentence': one_hot_corpus
}

df_one_hot = pd.DataFrame(data=data, index=corpus)
df_one_hot

,sentence_as_array,one_hot_sentence
Este es el primer documento.,"[este, es, el, primer, documento]","[[0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0],..."
Este documento es el segundo documento.,"[este, documento, es, el, segundo, documento]","[[0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0],..."
Este es el tercero.,"[este, es, el, tercero]","[[0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0],..."
¿Es este el primer documento?,"[es, este, el, primer, documento]","[[0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0],..."


Lo que obtenemos ahora es una matriz de cada una de las sentencias, en la cual estamos teniendo en cuenta el orden en el que aparecen las palabras dentro de la misma.

Frase 1:

In [ ]:
#@title
pd.DataFrame(index=df_one_hot.loc[corpus[0],'sentence_as_array'],
             data=df_one_hot.loc[corpus[0],'one_hot_sentence'],
             columns=words)

,documento,el,es,este,primer,segundo,tercero
este,0,0,0,1,0,0,0
es,0,0,1,0,0,0,0
el,0,1,0,0,0,0,0
primer,0,0,0,0,1,0,0
documento,1,0,0,0,0,0,0


Frase 2:

In [ ]:
#@title
pd.DataFrame(index=df_one_hot.loc[corpus[1],'sentence_as_array'],
             data=df_one_hot.loc[corpus[1],'one_hot_sentence'],
             columns=words)

,documento,el,es,este,primer,segundo,tercero
este,0,0,0,1,0,0,0
documento,1,0,0,0,0,0,0
es,0,0,1,0,0,0,0
el,0,1,0,0,0,0,0
segundo,0,0,0,0,0,1,0
documento,1,0,0,0,0,0,0


Para asegurar que las matrices asociadas a cada frase tienen el mismo número de filas, habría que rellenar con un placeholder (vector con todo 0's) aquellas frases de menor longitud tantas filas cómo sea la longitud de la frase más larga.
Para el caso anterior quedaría así:


In [ ]:
placeholder = pd.DataFrame({'*': [0, 0, 0, 0, 0, 0, 0]}, index=words).T

In [ ]:
#@title
pd.DataFrame(index=df_one_hot.loc[corpus[0],'sentence_as_array'],
             data=df_one_hot.loc[corpus[0],'one_hot_sentence'],
             columns=words).append(placeholder)

,documento,el,es,este,primer,segundo,tercero
este,0,0,0,1,0,0,0
es,0,0,1,0,0,0,0
el,0,1,0,0,0,0,0
primer,0,0,0,0,1,0,0
documento,1,0,0,0,0,0,0
*,0,0,0,0,0,0,0


## Index based encoding

Esta es quizás la manera más intuitiva de codificar una sentencia teniendo en cuenta el orden de las palabras.
Basta con asignar la palabra a su indice en el diccionario.

In [ ]:
pd.DataFrame(pd.Series(vectorizer.vocabulary_).sort_values(), columns=['indice'])

,indice
documento,0
el,1
es,2
este,3
primer,4
segundo,5
tercero,6


Quedaría así:


In [ ]:
index_based_corpus = np.array([np.array([vectorizer.vocabulary_[x] for x in corpus_as_array[i]]) for i in range(0, len(corpus_as_array))])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
#@title
data = {
    'sentence_as_array': corpus_as_array,
    'index_based_sentence': index_based_corpus
}

pd.DataFrame(data, index=corpus)

,sentence_as_array,index_based_sentence
Este es el primer documento.,"[este, es, el, primer, documento]","[3, 2, 1, 4, 0]"
Este documento es el segundo documento.,"[este, documento, es, el, segundo, documento]","[3, 0, 2, 1, 5, 0]"
Este es el tercero.,"[este, es, el, tercero]","[3, 2, 1, 6]"
¿Es este el primer documento?,"[es, este, el, primer, documento]","[2, 3, 1, 4, 0]"


Para asegurar que los vectores tengan el mismo tamaño se suele reservar el indice 0 del vocabulario (empezando este en 1) para rellenar el vector y completar hasta que todos los vectores sean de igual longitud.

El problema de esta codificación es que introduce una distancia numérica entre los textos que realmente no existe. Y por lo tanto no es muy recomendable su utilización.

## Word Embeddings

Word embeddings son un conjunto de técnicas de procesamiento del lenguaje natural que lo que permiten es mapear el significado semántico de las palabras en un espacio geométrico. Esto se consigue mediante la asociacióón de cada palabra de un diccionario a un vector. De tal manera que la distancia entre dos vectores cualquiera capture la relación semantica entre las dos palabras asociadas. El espacio geométrico formado por estos vectores se denomina embedding space. Las técnicas de word embedding más conocidas son Word2Vec y GloVe.

En la práctica, proyectamos cada palabra en un espacio vectorial continuo, producido por una capa de la red neuronal específica para ello. Este layer aprende a asociar una representación vectorial de cada palabra que es la mejor para completar su tarea general, por ejemplo, la predicción de una clase, la plabra siguiente, su traducción...


El embedding layer no es más que la proyección del vector one-hot encoded disperso, que hemos visto antes, en un espacio latente continuo y denso. Es una matriz de (n,m) donde n es el tamaño de su vocabulario y m son las dimensiones del espacio latente. Sólo que realmente, no hay necesidad de hacer la multiplicación de matrizes, y en su lugar se puede ahorrar el cálculo mediante el uso del índice de la palabra. Así que, en la práctica, es una capa que mapea enteros positivos (índices correspondientes a las palabras) en vectores densos de tamaño fijo (los vectores de embedding).

Se puede entrenar para crear una embedding Word2Vec utilizando Skip-Gram o CBOW(https://medium.com/swlh/a-quick-overview-of-the-main-difference-between-word2vec-and-fasttext-b9d3f6e274e9). O puedes entrenarlo para tu problema específico y obtener un embedding adecuado para la tarea que estás resolviendo. También puedes cargar un embedding pre-entrenado de los múltiples que existen y luego continuar el entrenamiento para tu problema (es una manera de aplicar transfer learning).

El problema principal de los embeddings es que no entienden el contexto de la palabra, es decir, existen palabras en español que dependiendo del contexto pueden significar una cosa u otra, por ejemplo, muñeca que puede ser "figura de persona, hecha generalmente de plástico, trapo o goma, que sirve de juguete o de adorno" o "parte del cuerpo humano en donde se articula la mano con el antebrazo".
Para un embedding en ambos casos va a significar lo mismo, aunque si el ámbito de tu problema es muy específico y las palabras no pueden aparecer en múltiples contextos no debería ser una preocupación.

https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526

In [ ]:
# Matriz de 5*7 (oración one-hot)
example = one_hot_corpus[0]
pd.DataFrame(example)

,0,1,2,3,4,5,6
0,0,0,0,1,0,0,0
1,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0
3,0,0,0,0,1,0,0
4,1,0,0,0,0,0,0


In [ ]:
# Matriz de 7*2 (matriz embedding)
M = np.array([[1, 1], [1, 2], [2, 2], [3,1], [1, 4], [1, 3], [3, 4]])
pd.DataFrame(M)

,0,1
0,1,1
1,1,2
2,2,2
3,3,1
4,1,4
5,1,3
6,3,4


In [ ]:
# Nueva representación de la oración
new_vector = np.matmul(example, M)
pd.DataFrame(new_vector)

,0,1
0,3,1
1,2,2
2,1,2
3,1,4
4,1,1


Para el ejemplo la matriz M la hemos calculado teniendo en cuenta que necesitabamos asignar a cada una de las 7 palabras una respresentación diferente, pero no es ni mucho menos la más optima. El embedding layer sí que calcula esta matriz óptima. Por lo tanto el resultado final del embedding después del entrenamiento sería la matriz M cuyos valores son más óptimos para resolver la tarea específica.

https://stackoverflow.com/questions/42762849/keras-embedding-layers-how-do-they-work

In [ ]:
pd.DataFrame(example) #5*7

,0,1,2,3,4,5,6
0,0,0,0,1,0,0,0
1,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0
3,0,0,0,0,1,0,0
4,1,0,0,0,0,0,0


In [ ]:
M = [['a', 'b'], ['c', 'd'], ['e', 'f'], ['g','h'], ['i', 'j'], ['k', 'l'], ['m', 'n']]
pd.DataFrame(M) #7*2

,0,1
0,a,b
1,c,d
2,e,f
3,g,h
4,i,j
5,k,l
6,m,n


In [ ]:
# Oración one hot * M = nuevo vector denso
new_vector = [['g','h'], ['e', 'f'], ['c', 'd'], ['i', 'j'], ['a', 'b']]
pd.DataFrame(new_vector) #5*2

,0,1
0,g,h
1,e,f
2,c,d
3,i,j
4,a,b


El valor de cada una de estas variables (a-n) es el que se calcula durante el entrenamiento. Nos va a permitir para el caso del ejemplo pasar de tener cada palabra en dimensión 7 a tenerla únicamente en 2 dimensiones.

El embedding layer de keras permite pasar de un vector de índices a tener directamente esta nueva codificación. Sin tener que pasarle las oraciones en one-hot.